`--- Day 8: Memory Maneuver ---`

In [1]:
import unittest
from collections import deque

The sleigh is much easier to pull than you'd expect for something its weight. Unfortunately, neither you nor the Elves know which way the North Pole is from here.

You check your wrist device for anything that might help. It seems to have some kind of navigation system! Activating the navigation system produces more bad news: "Failed to start navigation system. Could not read software license file."

The navigation system's license file consists of a list of numbers (your puzzle input). The numbers define a data structure which, when processed, produces some kind of tree that can be used to calculate the license number.

The tree is made up of nodes; a single, outermost node forms the tree's root, and it contains all other nodes in the tree (or contains nodes that contain nodes, and so on).

Specifically, a node consists of:
```
    A header, which is always exactly two numbers:
        The quantity of child nodes.
        The quantity of metadata entries.
    Zero or more child nodes (as specified in the header).
    One or more metadata entries (as specified in the header).
```
Each child node is itself a node that has its own header, child nodes, and metadata. For example:
```
2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2
A----------------------------------
    B----------- C-----------
                     D-----
```
In this example, each node of the tree is also marked with an underline starting with a letter for easier identification. In it, there are four nodes:
```
    A, which has 2 child nodes (B, C) and 3 metadata entries (1, 1, 2).
    B, which has 0 child nodes and 3 metadata entries (10, 11, 12).
    C, which has 1 child node (D) and 1 metadata entry (2).
    D, which has 0 child nodes and 1 metadata entry (99).
```
The first check done on the license file is to simply add up all of the metadata entries. In this example, that sum is 1+1+2+10+11+12+2+99=138.

**What is the sum of all metadata entries?**

In [2]:
license = deque(map(int, open('input.txt').read().strip().split(' ')))

In [3]:
class TestIt(unittest.TestCase):
    
    def setUp(self):
        test_license = deque(map(int, '2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2'.strip().split(' ')))
        self.tree = mkTree(test_license)
        
    def test_treeMetadata(self):
        self.assertEqual(self.tree.sumMetadata(), 138)
        
    def test_nodeCount(self):
        self.assertEqual(self.tree.nodeCount(), 4)

In [4]:
class Node(object):
    
    def __init__(self, n_children=0, n_metadata=0, parent=None, children=None, metadata=None):
        self.parent = parent
        self.n_children = n_children
        self.n_metadata = n_metadata
        self._value = None
        if children is None:
            self.children = []
        else:
            self.children = children[:] #  copy
        if metadata is None:
            self.metadata = []
        else:
            self.metadata = metadata[:] # copy
    
    def addChild(self, c):
        self.children.append(c)
        
    def addMetadata(self, i):
        self.metadata.append(i)
        return self
    
    def sumMetadata(self):
        metasum = sum(self.metadata)
        metasum += sum(c.sumMetadata() for c in self.children)
        return metasum
    
    def nodeCount(self):
        nodecount = 1 # self
        nodecount += sum(c.nodeCount() for c in self.children)
        return nodecount
    
    # part2
    def value(self):
        if self._value:
            return self._value
        elif self.n_children == 0:
            self._value = self.sumMetadata()
            return self._value
        else:
            self._value = 0
            for x in self.metadata:
                i = x - 1 # child refs start at 1
                if i < self.n_children:
                    self._value += self.children[i].value()
            return self._value
    
    def __str__(self):
        return f'Node(N_children={self.n_children}, n_metadata={self.n_metadata}, parent={self.parent}, children={self.children}, metadata={self.metadata})'
    
    __repr__ = __str__
    
def mkTree(lic, parent=None):
    n_children = lic.popleft()
    n_meta = lic.popleft()
    node = Node(n_children, n_meta, parent=parent)
    for i in range(n_children):
        c = mkTree(lic, node)
        node.addChild(c)
    for i in range(n_meta):
        node.addMetadata(lic.popleft())
    return node

In [5]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestIt)
unittest.TextTestRunner(verbosity=2).run(suite)

test_nodeCount (__main__.TestIt) ... ok
test_treeMetadata (__main__.TestIt) ... ok

----------------------------------------------------------------------
Ran 2 tests in 0.016s

OK


<unittest.runner.TextTestResult run=2 errors=0 failures=0>

# Part 1

In [6]:
tree = mkTree(license)

In [7]:
tree.sumMetadata()

46096

# --- Part Two ---

The second check is slightly more complicated: you need to find the value of the root node (A in the example above).

The value of a node depends on whether it has child nodes.

If a node has no child nodes, its value is the sum of its metadata entries. So, the value of node B is 10+11+12=33, and the value of node D is 99.

However, if a node does have child nodes, the metadata entries become indexes which refer to those child nodes. A metadata entry of 1 refers to the first child node, 2 to the second, 3 to the third, and so on. The value of this node is the sum of the values of the child nodes referenced by the metadata entries. If a referenced child node does not exist, that reference is skipped. A child node can be referenced multiple time and counts each time it is referenced. A metadata entry of 0 does not refer to any child node.

For example, again using the above nodes:

* Node C has one metadata entry, 2. Because node C has only one child node, 2 references a child node which does not exist, and so the value of node C is 0.
* Node A has three metadata entries: 1, 1, and 2. The 1 references node A's first child node, B, and the 2 references node A's second child node, C. Because node B has a value of 33 and node C has a value of 0, the value of node A is 33+33+0=66.

So, in this example, the value of the root node is 66.

What is the value of the root node?


In [8]:
class TestPart2(unittest.TestCase):
    
    def setUp(self):
        test_license = deque(map(int, '2 3 0 3 10 11 12 1 1 0 1 99 2 1 1 2'.strip().split(' ')))
        self.tree = mkTree(test_license)
        
    def test_treeValue(self):
        self.assertEqual(self.tree.value(), 66)

In [9]:
suite = unittest.TestLoader().loadTestsFromTestCase(TestPart2)
unittest.TextTestRunner(verbosity=2).run(suite)

test_treeValue (__main__.TestPart2) ... ok

----------------------------------------------------------------------
Ran 1 test in 0.000s

OK


<unittest.runner.TextTestResult run=1 errors=0 failures=0>

In [10]:
tree.value()

24820